In [2]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [5]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown for Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',  # Default value for the dropdown
        placeholder="Select a Launch Site here",  # Placeholder text
        searchable=True  # Enable search functionality
    ),
    html.Br(),

    # Pie chart for success count
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i} Kg' for i in range(0, 10001, 1000)},  # Marks at every 1000 Kg
        value=[min_payload, max_payload]  # Default range set to min and max payload
    ),
    html.Br(),

    # Scatter chart for payload vs success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Aggregate data to show total success launches by site
        site_counts = spacex_df[spacex_df['class'] == 1].groupby('Launch Site').size().reset_index(name='count')
        fig = px.pie(
            site_counts,
            values='count',
            names='Launch Site',
            title='Total Success Launches By Site'
        )
    else:
        # Filter dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_counts = filtered_df['class'].value_counts()
        fig = px.pie(
            names=['Success', 'Failure'],
            values=site_counts,
            title=f'Total Success Launches for Site {entered_site}'
        )
    return fig

# Callback for the scatter plot
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if selected_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Correlation between Payload and Success for All Sites',
            labels={'class': 'Launch Success'}
        )
    else:
        site_data = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_data,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Correlation between Payload and Success for {selected_site}',
            labels={'class': 'Launch Success'}
        )
    return fig

# Display in different tabs
@app.callback(Output('tab-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Tab content 1')
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])


# Run the app
if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>

# 1. Which site has the largest successful launches?

In [6]:
spacex_df[spacex_df['class'] == 1].groupby('Launch Site').size().idxmax()


'KSC LC-39A'

# 2. Which site has the highest launch success rate?

In [7]:
site_success_rate = spacex_df.groupby('Launch Site')['class'].mean()
site_success_rate.idxmax()  # Site with the highest success rate


'KSC LC-39A'

# 3. Which payload range(s) has the highest launch success rate?

In [8]:
spacex_df['Payload Range'] = pd.cut(
    spacex_df['Payload Mass (kg)'], bins=[0, 2000, 4000, 6000, 8000, 10000], labels=['0-2k', '2k-4k', '4k-6k', '6k-8k', '8k-10k']
)
payload_success_rate = spacex_df.groupby('Payload Range')['class'].mean()
payload_success_rate.idxmax()  # Range with the highest success rate


<ipython-input-8-afd3c0a4d879>:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



'2k-4k'

# 4. Which payload range(s) has the lowest launch success rate?

In [10]:
payload_success_rate.idxmin()  # Range with the lowest success rate


'6k-8k'

# 5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

In [11]:
booster_success_rate = spacex_df.groupby('Booster Version Category')['class'].mean()
booster_success_rate.idxmax()  # Booster version with the highest success rate


'B5'